# Get Sensors Reading

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json
from datetime import datetime,timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

In [3]:
# select parameters
building_id=2300
#start_time='2025-06-01T00:00:00'
#end_time='2025-06-30T23:59:00'

# number of days before now
x_days = 7  # number of days before now
now = datetime.now()
start_datetime = now - timedelta(days=x_days)
start_time = start_datetime.strftime('%Y-%m-%dT%H:%M:%S')
end_time = now.strftime('%Y-%m-%dT%H:%M:%S')

#granularity
granularity_in_seconds=60

## Functions

### sensor list for building

In [4]:
def get_sensor_dataframe_by_building_id(building_id, client):
    """
    Fetch sensor data for a given building ID and return it as a pandas DataFrame,
    with sensor ID as the index and relevant metadata as columns.
    
    Parameters:
    - building_id (int): The building ID.
    - client: The API client with the `buildings` attribute.
    
    Returns:
    - pd.DataFrame: DataFrame with sensor metadata.
    """
    sensors = client.buildings.get_list_of_sensors_by_building_id(building_id)
    
    data = []
    for sensor in sensors:
        data.append({
            "sensor_id": sensor.id,
            "unit": sensor.unit,
            "factor": sensor.factor,
            "acronym": sensor.acronym,
            "hydraulic_location_index": sensor.hydraulic_location_index,
            "long_name": sensor.long_name,
            "short_name": sensor.short_name,
            "source":sensor.source
        })
    
    df = pd.DataFrame(data).set_index("sensor_id")
    # Drop rows where unit, long_name, and short_name are all None
    df = df[~(df["unit"].isna() & df["long_name"].isna() & df["short_name"].isna())]
    return df

In [5]:
#client.buildings.get_list_of_sensors_by_building_id(2300)

### Sensors readings

In [6]:
def get_chart_data_with_building_str(sensor_ids, start_time, end_time, granularity_in_seconds, client):
    response = client.charts.data(sensor_ids, start_time, end_time, granularity_in_seconds)
    
    sensor_value_dfs = []
    building_id = None

    for sensor_id_str, datapoints in response.sensors.items():
        if not datapoints:
            continue
        
        if building_id is None:
            building_id = str(datapoints[0].building)  # get building from first datapoint
        
        times = [dp.time for dp in datapoints]
        values = [dp.value for dp in datapoints]

        df_values = pd.DataFrame({
            "timestamp": pd.to_datetime(times),
            sensor_id_str: values
        }).set_index("timestamp")

        sensor_value_dfs.append(df_values)

    df_values_combined = pd.concat(sensor_value_dfs, axis=1).sort_index()

    return df_values_combined
#client.charts.data(sensor_ids,start_time,end_time,granularityInSeconds)

### Sensor infos

## get data

In [7]:
df_sensors_infos = get_sensor_dataframe_by_building_id(building_id, client)
df_sensors_infos.index = df_sensors_infos.index.astype(int)

In [8]:
df_sensors_infos

,unit,factor,acronym,hydraulic_location_index,long_name,short_name,source
sensor_id,,,,,,,
37006,°C,0.1,t_b(n)_flow,1.0,Vorlauftemperatur Kessel (n),T VL K (n),GREENBOX_MQTT
37008,°C,0.1,t_b(n)_return,1.0,Rücklauftemperatur Kessel (n),T RL K (n),GREENBOX_MQTT
37010,°C,0.1,t_hc(n)_flow,1.0,Vorlauftemperatur Heizkreis (n),T VL HK (n),GREENBOX_MQTT
37012,°C,0.1,t_hc(n)_return,1.0,Rücklauftemperatur Heizkreis (n),T RL HK (n),GREENBOX_MQTT
37018,°C,0.1,t_hw(n)_pwh,1.0,Temperatur warmes Trinkwasser Warmwasser (n),T TWwarm WW (n),GREENBOX_MQTT
37020,°C,0.1,t_hw(n)_circ,1.0,Temperatur Zirkulation Warmwasser (n),T ZK WW (n),GREENBOX_MQTT
37022,°C,1.0,t_g_out_api,NaN,Außentemperatur Global,T A GL,OPEN_WEATHER_API
37014,°C,0.1,t_hw(n)_return,1.0,Rücklauftemperatur Speicherladung Warmwasser (n),T RLload WW (n),GREENBOX_MQTT
37016,°C,0.1,t_hw(n)_tank,1.0,Temperatur Trinkwasserspeicher Warmwasser (n),T SP WW (n),GREENBOX_MQTT


In [9]:
#exclude manually entered
df_sensors_infos = df_sensors_infos[df_sensors_infos['source'] != 'MANUALLY_ENTERED']

In [10]:
#list(df_sensors.index)

In [11]:
df_sensor_data = get_chart_data_with_building_str(
    sensor_ids=list(df_sensors_infos.index),
    start_time=start_time,
    end_time=end_time,
    granularity_in_seconds=granularity_in_seconds,
    client=client
)
df_sensor_data.columns = df_sensor_data.columns.astype(int)

In [12]:
df_sensor_data.head()

,37006,37008,37010,37012,37014,37016,37018,37020,37022,37025
timestamp,,,,,,,,,,
2025-07-11 16:45:00+00:00,56.4,46.2,30.7,29.8,47.1,60.7,59.5,50.9,16.33,NaN
2025-07-11 16:46:00+00:00,56.3,46.2,30.7,29.7,47.0,60.6,59.5,50.8,NaN,NaN
2025-07-11 16:47:00+00:00,56.2,46.1,30.7,29.7,46.9,60.6,59.4,50.9,NaN,NaN
2025-07-11 16:48:00+00:00,56.2,46.1,30.7,29.7,46.8,60.6,59.4,50.8,16.99,NaN
2025-07-11 16:49:00+00:00,56.1,46.1,30.7,29.7,46.7,60.5,59.3,50.7,NaN,NaN


## Charts

In [13]:
def plot_sensors(df_sensor_data, df_sensors_infos, html_filename):
    # Get unique units
    units = df_sensors_infos['unit'].unique()
    
    # Sort units: °C first, then kWh, then others
    def unit_sort_key(u):
        if u == '°C':
            return 0
        elif u.lower() == 'kwh':
            return 1
        else:
            return 2
    sorted_units = sorted(units, key=unit_sort_key)
    
    n_units = len(sorted_units)
    
    base_height_per_unit = 400
    vertical_spacing = 0.1
    
    fig_height = int(base_height_per_unit * n_units * 1.3)  # increase height by 30%
    fig_width = 1600
    
    fig = make_subplots(
        rows=n_units,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=vertical_spacing,
        subplot_titles=[f"Unit: {unit}" for unit in sorted_units]
    )
    # Show x-axis tick labels on all subplots
    for i in range(1, n_units + 1):
        fig.update_xaxes(showticklabels=True, row=i, col=1)
    
    # Iterate over units with their subplot row index
    for i, unit in enumerate(sorted_units, start=1):
        # Get sensor IDs that belong to this unit
        sensor_ids = df_sensors_infos[df_sensors_infos['unit'] == unit].index
        
        for sensor_id in sensor_ids:
            if sensor_id not in df_sensor_data.columns:
                continue  # no data
            
            y_data = df_sensor_data[sensor_id].dropna()  # filter here
            
            if y_data.empty:
                continue  # skip if no valid data
            
            fig.add_trace(
                go.Scatter(
                    x=y_data.index,    # use filtered index
                    y=y_data,          # use filtered data without NaNs
                    mode='lines',
                    name=f"{sensor_id} {df_sensors_infos.loc[sensor_id, 'acronym']}"
                ),
                row=i,
                col=1
            )
        

        fig.update_xaxes(
            showticklabels=True,
            row=i,
            col=1
        )
    
    fig.update_layout(
        margin=dict(l=60, r=60, t=80, b=60),
    )
    
    fig.write_html(html_filename)
    print(f"Plot saved as {html_filename}")


In [14]:
file_name='sensors_of_building_'+str(building_id)+'.html'
plot_sensors(df_sensor_data, df_sensors_infos, file_name)

Plot saved as sensors_of_building_2300.html
